In [23]:
import pandas as pd
import numpy as np
import os
import zipfile
import json
from datetime import datetime
import time
import psycopg2
import sqlalchemy
from psycopg2 import sql
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Float, String, DateTime, Text
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

start_time = time.time()

In [24]:
dataset_sources = [
    ("2023", "http://dados.recife.pe.gov.br/dataset/28e3e25e-a9a7-4a9f-90a8-bb02d09cbc18/resource/a01e3b82-055b-4139-b495-e54ec5196f54/download/itbi_2023.csv"),
    ("2024", "http://dados.recife.pe.gov.br/dataset/28e3e25e-a9a7-4a9f-90a8-bb02d09cbc18/resource/c3feb7bb-4a76-4ddf-becb-ea77f66c23d0/download/itbi_2024.csv"),
    ("2025", "http://dados.recife.pe.gov.br/dataset/28e3e25e-a9a7-4a9f-90a8-bb02d09cbc18/resource/97bbfc62-efb2-493f-9631-09eb05f980d7/download/itbi_2025.csv")
]

datasets_dict = {}
total_records = 0

print("CARREGANDO DADOS ITBI - RECIFE")
print("=" * 30)

for year, url in dataset_sources:
    try:
        print(f"{year}: ", end="")
        df = pd.read_csv(url, sep=';', encoding='utf-8')
        df['year'] = int(year)
        datasets_dict[year] = df
        total_records += len(df)
        print(f"OK - {len(df):,} registros")
    except Exception as e:
        print(f"Erro: {e}")

print(f"\nRESUMO:")
print(f"  Total: {total_records:,} registros")
print(f"  Anos: {list(datasets_dict.keys())}")
print(f"  Colunas: {len(datasets_dict["2023"].columns)}")


CARREGANDO DADOS ITBI - RECIFE
2023: OK - 12,670 registros
2024: OK - 15,243 registros
2025: OK - 10,020 registros

RESUMO:
  Total: 37,933 registros
  Anos: ['2023', '2024', '2025']
  Colunas: 24


In [25]:
#Preview dos dados
print(f"\nAMOSTRA DOS DADOS:")
datasets_dict['2023'][['bairro', 'tipo_imovel', 'valor_avaliacao']].head(3)


AMOSTRA DOS DADOS:


,bairro,tipo_imovel,valor_avaliacao
0,Encruzilhada,Galpão,1068562.63
1,Encruzilhada,Casa,1500000.00
2,Encruzilhada,Apartamento,110000.00


In [26]:
#CONEXÃO COM POSTGRESQL
#======================
#Carregar configuração do banco de dados
with open('config.json', 'r') as config_file:
    db_config = json.load(config_file)

print("CONECTANDO AO POSTGRESQL")
print("=" * 24)
print(f"  Host: {db_config['host']}")
print(f"  Porta: {db_config['port']}")
print(f"  Banco: {db_config['database']}")
print(f"  Usuário: {db_config['user']}")

try:
    # Testar conexão com psycopg2
    conn = psycopg2.connect(
        host=db_config['host'],
        port=db_config['port'],
        database=db_config['database'],
        user=db_config['user'],
        password=db_config['password']
    )

#Criar engine SQLAlchemy para pandas
    engine = create_engine(
        f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
    )

#Verificar detalhes do banco
    cursor = conn.cursor()

#Versão do PostgreSQL
    cursor.execute("SELECT version();")
    version = cursor.fetchone()[0]

#Verificar se existem tabelas
    cursor.execute("SELECT count(*) FROM information_schema.tables WHERE table_schema = 'public';")
    table_count = cursor.fetchone()[0]

    print(f"\nCONEXÃO ESTABELECIDA!")
    print(f"  Versão: {version.split()[0]} {version.split()[1]}")
    print(f"  Tabelas existentes: {table_count}")
    print(f"  Engine SQLAlchemy: Pronto")

#Fechar cursor mas manter conexões para uso posterior
    cursor.close()

    print(f"\nBanco PostgreSQL pronto para receber dados ITBI!")

except Exception as e:
    print(f"\nERRO NA CONEXÃO: {e}")
    print(f"\nVERIFICAÇÕES:")
    print(f"  - PostgreSQL está rodando?")
    print(f"  - Banco '{db_config['database']}' existe?")
    print(f"  - Credenciais corretas no config.json?")
    print(f"  - Porta {db_config['port']} disponível?")


CONECTANDO AO POSTGRESQL
  Host: localhost
  Porta: 5432
  Banco: elt_pipeline
  Usuário: postgres

CONEXÃO ESTABELECIDA!
  Versão: PostgreSQL 16.9
  Tabelas existentes: 4
  Engine SQLAlchemy: Pronto

Banco PostgreSQL pronto para receber dados ITBI!


In [29]:
#INSERÇÃO NO POSTGRESQL
#======================
print("Inserindo dados ITBI...")

try:
    # Inserir cada ano
    for year, df in datasets_dict.items():
        df.to_sql(f"itbi_{year}", engine, if_exists='replace', index=False, method='multi')
        print(f"OK - itbi_{year}: {len(df):,} registros")

    #Tabela consolidada
    with engine.connect() as conn:
        conn.execute(sqlalchemy.text("""
            CREATE SCHEMA IF NOT EXISTS raw_itbi;
            DROP TABLE IF EXISTS raw_itbi.itbi_consolidado;
            CREATE TABLE raw_itbi.itbi_consolidado AS
            SELECT *, '2023' as fonte FROM itbi_2023
            UNION ALL 
            SELECT *, '2024' as fonte FROM itbi_2024
            UNION ALL 
            SELECT *, '2025' as fonte FROM itbi_2025;
        """))
        conn.commit()

    total = pd.read_sql("SELECT COUNT(*) as total FROM raw_itbi.itbi_consolidado", engine).iloc[0]['total']
    print(f"OK - itbi_consolidado: {total:,} registros")
    print("Concluído!")

except Exception as e:
    print(f"Erro: {e}")

Inserindo dados ITBI...
OK - itbi_2023: 12,670 registros
OK - itbi_2024: 15,243 registros
OK - itbi_2025: 10,020 registros
OK - itbi_consolidado: 37,933 registros
Concluído!
